In [ ]:
import lightgbm as lgb
from fbprophet import Prophet, make_holidays
import pandas as pd
from keras.layers import Dense, LSTM, Dropout
from keras.models import Sequential
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 数据路径
data_file = '../input/hydrowater/data'
# read data
Qidata = data_file + '/Qi.xlsx'
Raindata = data_file + '/Rain_forecast.xlsx'
Environmentdata = data_file + '/Env.xlsx'
RemoteRaindata = data_file + '/Remote_rain.xlsx'


# 数据路径
data_file_final = '../input/hydrowater'
# read data
Qidata_final = data_file_final + '/Qi_final.xlsx'
Raindata_final = data_file_final + '/Rain_forecast_final.xlsx'
Environmentdata_final = data_file_final + '/Env_final.xlsx'
RemoteRaindata_final = data_file_final + '/Remote_rain_final.xlsx'

In [ ]:
# 读入数据，用TimeStample 作为index，并且转换为时间戳格式
qi_raw_df = pd.read_excel(Qidata, index_col='TimeStample', parse_dates=True)
rain_raw_df = pd.read_excel(
    Raindata,
    index_col='TimeStample',
    parse_dates=True)
env_raw_df = pd.read_excel(
    Environmentdata,
    index_col='TimeStample',
    parse_dates=True)
remote_rain_df = pd.read_excel(
    RemoteRaindata,
    index_col='TimeStample',
    parse_dates=True)

qi_raw_df_final = pd.read_excel(Qidata_final, index_col='TimeStample', parse_dates=True)
rain_raw_df_final = pd.read_excel(
    Raindata_final,
    index_col='TimeStample',
    parse_dates=True)
env_raw_df_final = pd.read_excel(
    Environmentdata_final,
    index_col='TimeStample',
    parse_dates=True)
remote_rain_df_final = pd.read_excel(
    RemoteRaindata_final,
    index_col='TimeStample',
    parse_dates=True)
rain_raw_df_final.drop('5天合计',inplace=True,axis=1)



# resample and fillna 重采样，fillna
rain_raw_df.loc[pd.Timestamp('2018-11-08')
] = rain_raw_df.loc[pd.Timestamp('2018-11-07')]  # 重采样会放弃2018-11-07的样本，所以在此加上2018-11-08
env_raw_df.loc[pd.Timestamp('2018-11-01')
] = env_raw_df.loc[pd.Timestamp('2018-10-31')]  # 重采样会放弃2018-10-31的样本，所以在此加上2018-11-01
remote_rain_df.loc[pd.Timestamp('2018-11-01')
] = remote_rain_df.loc[pd.Timestamp('2018-10-31')]  # 重采样会放弃2018-10-31 23:00的样本，所以在此加上2018-11-01
remote_rain_df['rain_sum'] = remote_rain_df.sum(axis=1)  # 遥感数据求和
remote_rain_resample = remote_rain_df[['rain_sum']].resample(rule='3H').sum()  # 遥感数据重采样
rain_raw_resample = rain_raw_df.resample(rule='3H').interpolate('linear') / 8  # 雨量预测重采样
env_raw_resample = env_raw_df.resample(rule='3H').interpolate('linear')  # 环境重采样
env_rain = pd.concat(
    (rain_raw_resample,
     env_raw_resample,
     remote_rain_resample),
    axis=1)  # 所有重采样的数据合并
env_rain.tail(66)

env_rain.index = env_rain.index - pd.Timedelta(value=1, unit='H')  # 所有重采样的数据合并后，调整时间戳


# resample and fillna 重采样，fillna
rain_raw_df_final.loc[pd.Timestamp('2019-11-08')
] = rain_raw_df_final.loc[pd.Timestamp('2019-11-07')]  # 重采样会放弃2018-11-07的样本，所以在此加上2018-11-08
env_raw_df_final.loc[pd.Timestamp('2019-11-01')
] = env_raw_df_final.loc[pd.Timestamp('2019-10-31')]  # 重采样会放弃2018-10-31的样本，所以在此加上2018-11-01
remote_rain_df_final.loc[pd.Timestamp('2019-11-01')
] = remote_rain_df_final.loc[pd.Timestamp('2019-10-31')]  # 重采样会放弃2018-10-31 23:00的样本，所以在此加上2018-11-01
remote_rain_df_final['rain_sum'] = remote_rain_df_final.sum(axis=1)  # 遥感数据求和
remote_rain_resample_final = remote_rain_df_final[['rain_sum']].resample(rule='3H').sum()  # 遥感数据重采样
rain_raw_resample_final = rain_raw_df_final.resample(rule='3H').interpolate('linear') / 8  # 雨量预测重采样
env_raw_resample_final = env_raw_df_final.resample(rule='3H').interpolate('linear')  # 环境重采样
env_rain_final = pd.concat(
    (rain_raw_resample_final,
     env_raw_resample_final,
     remote_rain_resample_final),
    axis=1)  # 所有重采样的数据合并
env_rain_final.tail(66)

env_rain_final.index = env_rain_final.index - pd.Timedelta(value=1, unit='H')  # 所有重采样的数据合并后，调整时间戳

In [ ]:
raw_all_df = pd.concat((qi_raw_df, env_rain), axis=1)  # 合并重采样数据和流量
raw_all_df_final = pd.concat((qi_raw_df_final, env_rain_final), axis=1)  # 合并重采样数据和流量

raw_all_df = pd.concat((raw_all_df, raw_all_df_final), axis=0)

In [ ]:
pf_df = raw_all_df.copy()
# predict rain_sum
from sklearn.linear_model import LinearRegression

pf_df['rain_last'] = pf_df['rain_sum'].shift(1)
pf_df['D1_last'] = pf_df['D1'].shift(1)
train_df = pf_df[['rain_sum', 'D1_last', 'rain_last']].dropna(axis=0).copy()
X = train_df[['D1_last', 'rain_last']].values
y = train_df['rain_sum'].values
lr_rain = LinearRegression().fit(X, y)
test_df = pf_df[['D1_last', 'rain_last']].dropna(axis=0).copy()

train_df['rain_pred'] = lr_rain.predict(X)
pf_df = pd.concat((pf_df, train_df[['rain_pred']]), axis=1)


def recursive_forecast(model, test_X, fill_col, X_cols, y_col, t_delta='3H', ):
    test_X_1 = test_X.copy()
    for next_x in test_X_1.index:
        pred = model.predict(test_X_1.loc[[next_x], X_cols].values)
        if next_x + pd.Timedelta(t_delta) <= test_X_1.index.max():
            test_X_1.loc[[next_x + pd.Timedelta(t_delta)], [fill_col]] = pred
    test_X_1[y_col] = model.predict(test_X_1[X_cols].values)
    return test_X_1


# print(pf_df['2018-11-01':'2018-11-07'])
pf_df.loc['2018-11-01':'2018-11-07'] = recursive_forecast(lr_rain, pf_df.loc['2018-11-01':'2018-11-07', ],
                                                          fill_col='rain_last', X_cols=['D1_last', 'rain_last'],
                                                          y_col='rain_pred', t_delta='3H', )
pf_df.loc['2018-02-01':'2018-02-07'] = recursive_forecast(lr_rain, pf_df.loc['2018-02-01':'2018-02-07', ],
                                                          fill_col='rain_last', X_cols=['D1_last', 'rain_last'],
                                                          y_col='rain_pred', t_delta='3H', )
pf_df.loc['2018-08-01':'2018-08-07'] = recursive_forecast(lr_rain, pf_df.loc['2018-08-01':'2018-08-07', ],
                                                          fill_col='rain_last', X_cols=['D1_last', 'rain_last'],
                                                          y_col='rain_pred', t_delta='3H', )
pf_df.loc['2019-02-01':'2019-02-07'] = recursive_forecast(lr_rain, pf_df.loc['2019-02-01':'2019-02-07', ],
                                                          fill_col='rain_last', X_cols=['D1_last', 'rain_last'],
                                                          y_col='rain_pred', t_delta='3H', )
pf_df.loc['2019-04-01':'2019-04-07'] = recursive_forecast(lr_rain, pf_df.loc['2019-04-01':'2019-04-07', ],
                                                          fill_col='rain_last', X_cols=['D1_last', 'rain_last'],
                                                          y_col='rain_pred', t_delta='3H', )
pf_df.loc['2019-06-01':'2019-06-07'] = recursive_forecast(lr_rain, pf_df.loc['2019-06-01':'2019-06-07', ],
                                                          fill_col='rain_last', X_cols=['D1_last', 'rain_last'],
                                                          y_col='rain_pred', t_delta='3H', )
pf_df.loc['2019-08-01':'2019-08-07'] = recursive_forecast(lr_rain, pf_df.loc['2019-08-01':'2019-08-07', ],
                                                          fill_col='rain_last', X_cols=['D1_last', 'rain_last'],
                                                          y_col='rain_pred', t_delta='3H', )
pf_df.loc['2019-11-01':'2019-11-07'] = recursive_forecast(lr_rain, pf_df.loc['2019-11-01':'2019-11-07', ],
                                                          fill_col='rain_last', X_cols=['D1_last', 'rain_last'],
                                                          y_col='rain_pred', t_delta='3H', )




print(pf_df.tail(60))

In [ ]:
print(pf_df['rain_pred'].tail(60))
# pf_df[pf_df[['rain_pred','rain_last']].isna()][['rain_pred']]
print(pf_df[['rain_pred']])


In [ ]:
# 采用prophet 预测
# feature engineering
pf_df['future_5D_rain_sum'] = pf_df[[
    'D1', 'D2', 'D3', 'D4', 'D5']].sum(axis=1)  # 预测数，5天求和
# D1-D5, create long lag because it has data on test data set
pf_df[['D1_x_lag']] = pf_df[['D1']].rolling(102).mean()
pf_df[['D2_x_lag']] = pf_df[['D2']].rolling(110).mean()
pf_df[['D3_x_lag']] = pf_df[['D3']].rolling(108).mean()
pf_df[['D4_x_lag']] = pf_df[['D4']].rolling(123).mean()
pf_df[['D5_x_lag']] = pf_df[['D5']].rolling(134).mean()
pf_df['future_5D_rain_sum_x_lag'] = pf_df[['future_5D_rain_sum']].rolling(100).mean()
pf_df[['rain_pred_x_lag']] = pf_df[['rain_pred']].rolling('11D6H').mean()

target = 'Qi'
# cols = [
#     target,
#     'T',
#     'rain_sum',
#     'w',
#     'wd','D1', 'D2', 'D3', 'D4', 'D5','future_5D_rain_sum'
# ]
# non_use_cols = ['ds'] + cols

# def create_lags(df, cols, lag, base=1, suffix=None):
#     if suffix is None:
#         suffix = '_' + str(lag) + 'days'
#     for col in cols:
#         df[col + suffix] = df[col].shift(lag * base)
#     return df

print(pf_df.tail(60))

In [ ]:
def fillna_to_outliers(df, col, window='1D', threshold=0.0):
    std = df[col].rolling(window=window).std()
    std.fillna(0, inplace=True)
    df.loc[df[std <= threshold].index, col] = None
    return df


print(pf_df.isna().sum())
pf_df1 = fillna_to_outliers(pf_df, 'Qi', window='12H')
print(pf_df1.isna().sum())
print(pf_df1.tail())
pf_df1['Qi'].plot()

In [ ]:
pf_df['ds'] = pf_df.index  # prophet 需要命名ds列
pf_df.rename({'Qi': 'y'}, axis=1, inplace=True)  # prophet 需要命名独的y列

print(pf_df.columns)
pf_df = pf_df.sort_values(by=['ds'])
pf_df['cap'] = 0.9
pf_df['floor'] = 0.01

pf_df['on_season'] = (pf_df['ds'].dt.month > 3) & (pf_df['ds'].dt.month < 12)
pf_df['off_season'] = ~(pf_df['ds'].dt.month > 3) & (pf_df['ds'].dt.month < 12)

pf_df['D1_on'] = pf_df['on_season'] * pf_df['D1_x_lag']
pf_df['D1_off'] = pf_df['off_season'] * pf_df['D1_x_lag']
pf_df['D2_on'] = pf_df['on_season'] * pf_df['D2_x_lag']
pf_df['D2_off'] = pf_df['off_season'] * pf_df['D2_x_lag']
pf_df['D3_on'] = pf_df['on_season'] * pf_df['D3_x_lag']
pf_df['D3_off'] = pf_df['off_season'] * pf_df['D3_x_lag']
pf_df['D4_on'] = pf_df['on_season'] * pf_df['D4_x_lag']
pf_df['D4_off'] = pf_df['off_season'] * pf_df['D4_x_lag']
pf_df['D5_on'] = pf_df['on_season'] * pf_df['D5_x_lag']
pf_df['D5_off'] = pf_df['off_season'] * pf_df['D5_x_lag']
pf_df['rain_pred_on'] = pf_df['on_season'] * pf_df['rain_pred_x_lag']
pf_df['rain_pred_off'] = pf_df['off_season'] * pf_df['rain_pred_x_lag']

pf_df.describe()
start_dt = pd.Timestamp('2013-04-01 00:00:00')
end_dt = pd.Timestamp('2019-11-07 23:00:00')
train_df = pf_df[(pf_df['ds'] <= end_dt) & (pf_df['ds'] >= start_dt)]  # .dropna(axis=0)

# pf_use_cols = ['ds','cap','floor','y','T','w','wd','rain_sum','D1','D2','D3','D4','D5','future_5D_rain_sum']
pf_use_cols = ['ds', 'cap', 'floor', 'y', 'D1_x_lag', 'D2_x_lag', 'D3_x_lag', 'D4_x_lag', 'D5_x_lag', 'on_season',
               'off_season', 'D1_on', 'D2_on', 'D3_on', 'D4_on', 'D5_on', 'D1_off', 'D2_off', 'D3_off', 'D4_off',
               'D5_off', 'rain_pred_on', 'rain_pred_off',
               'rain_pred_x_lag']  # ,'future_5D_rain_sum','T_x_lag','w_x_lag','rain_sum_x_lag']
# pf_use_cols = pf_df.columns
pf_model = Prophet(yearly_seasonality=False,
                   weekly_seasonality=False,
                   daily_seasonality=False, growth='logistic')
pf_model.add_regressor('D1_x_lag')
pf_model.add_regressor('D2_x_lag')
pf_model.add_regressor('D3_x_lag')
pf_model.add_regressor('D4_x_lag')
pf_model.add_regressor('D5_x_lag')
pf_model.add_regressor('rain_pred_x_lag')
pf_model.add_regressor('rain_pred_on')
pf_model.add_regressor('rain_pred_off')
pf_model.add_regressor('D1_on')
pf_model.add_regressor('D2_on')
pf_model.add_regressor('D3_on')
pf_model.add_regressor('D4_on')
pf_model.add_regressor('D5_on')
pf_model.add_regressor('D1_off')
pf_model.add_regressor('D2_off')
pf_model.add_regressor('D3_off')
pf_model.add_regressor('D4_off')
pf_model.add_regressor('D5_off')
pf_model.add_seasonality(name='year_on', period=365.25, fourier_order=24, prior_scale=10, condition_name='on_season')
pf_model.add_seasonality(name='year_off', period=365.25, fourier_order=24, prior_scale=10, condition_name='off_season')

pf_model.add_seasonality(name='week_on', period=7, fourier_order=24, prior_scale=10, condition_name='on_season')
pf_model.add_seasonality(name='week_off', period=7, fourier_order=24, prior_scale=10, condition_name='off_season')

pf_model.add_seasonality(name='day_on', period=1, fourier_order=24, prior_scale=10, condition_name='on_season')
pf_model.add_seasonality(name='day_off', period=1, fourier_order=24, prior_scale=10, condition_name='off_season')

pf_model.add_seasonality(name='month_on', period=30, fourier_order=24, prior_scale=10, condition_name='on_season')
pf_model.add_seasonality(name='month_off', period=30, fourier_order=24, prior_scale=10, condition_name='off_season')
pf_model.add_seasonality(name='6_month_on', period=182.625, fourier_order=24, prior_scale=10,
                         condition_name='on_season')
pf_model.add_seasonality(name='6_month_off', period=182.625, fourier_order=24, prior_scale=10,
                         condition_name='off_season')

pf_model.fit(train_df[pf_use_cols])

In [ ]:
# 将prophet的预测作为一个特征，后面决定是否需要采用，接下来需要采用lightgbm
lgb_df = pf_df.copy()
pf_check_nan_cols = pf_use_cols[4:]
pf_use_cols[4:]
pf_predict_all_df = pf_df[pf_check_nan_cols].dropna(axis=0)
y_hat = pf_model.predict(lgb_df.loc[pf_predict_all_df.index])
y_hat.set_index('ds', inplace=True)
lgb_df.rename({'y': 'Qi'}, axis=1, inplace=True)
lgb_df = pd.concat((lgb_df, y_hat[['yhat']]), axis=1)
print(lgb_df['yhat'].describe())
print(lgb_df['yhat'].tail(66))

In [ ]:
lgb_df['yhat'][lgb_df['yhat'] <= 0.001] = 0.001
lgb_df['yhat'][lgb_df['yhat'] >= 0.99] = 0.99

In [ ]:
fg, ax = plt.subplots()
# ax.plot(lgb_df.loc['2013-10':'2014-02',['D1_x_lag']].values)
# ax.plot(lgb_df.loc['2014-10':'2015-02',['D1_x_lag']].values)
# ax.plot(lgb_df.loc['2015-10':'2016-02',['D1_x_lag']].values)
ax.plot(lgb_df.loc['2016-10':'2017-02', ['D1_x_lag']].values)

plt.show()
ax.set_ylim(0, 0.5)

fg, ax1 = plt.subplots()
# ax1.plot(lgb_df.loc['2013-10':'2014-02',['Qi']].values)
# ax1.plot(lgb_df.loc['2014-10':'2015-02',['Qi']].values)
# ax1.plot(lgb_df.loc['2015-10':'2016-02',['Qi']].values)
ax1.plot(lgb_df.loc['2016-10':'2017-02', ['Qi']])

plt.show()


# print(lgb_df.head(10))
# print(lgb_df.tail(10))

In [ ]:
def nes_score1(y_ture, y_pred):
    nes_score = 1 - np.sum((y_ture - y_pred) ** 2) / np.sum((y_ture - np.mean(y_ture)) ** 2)
    score = nes_score * 100
    return score


r = 10
r_s = lgb_df[['Qi', 'yhat']].copy().dropna(axis=0)

plt.plot(r_s['yhat'].values[-56:])
plt.plot(r_s['Qi'].values[-56:])
print(nes_score1(r_s['Qi'].values, r_s['yhat'].values))
print('01')
print(nes_score1(r_s.loc['2014-01', 'Qi'].values, r_s.loc['2014-01', 'yhat'].values))
print(nes_score1(r_s.loc['2015-01', 'Qi'].values, r_s.loc['2015-01', 'yhat'].values))
print(nes_score1(r_s.loc['2016-01', 'Qi'].values, r_s.loc['2016-01', 'yhat'].values))
print(nes_score1(r_s.loc['2017-01', 'Qi'].values, r_s.loc['2017-01', 'yhat'].values))
print(nes_score1(r_s.loc['2018-01', 'Qi'].values, r_s.loc['2018-01', 'yhat'].values))
print(nes_score1(r_s.loc['2019-01', 'Qi'].values, r_s.loc['2019-01', 'yhat'].values))

print('02')
print(nes_score1(r_s.loc['2014-02', 'Qi'].values, r_s.loc['2014-02', 'yhat'].values))
print(nes_score1(r_s.loc['2015-02', 'Qi'].values, r_s.loc['2015-02', 'yhat'].values))
print(nes_score1(r_s.loc['2016-02', 'Qi'].values, r_s.loc['2016-02', 'yhat'].values))
print(nes_score1(r_s.loc['2017-02', 'Qi'].values, r_s.loc['2017-02', 'yhat'].values))

print('03')
print(nes_score1(r_s.loc['2014-03', 'Qi'].values, r_s.loc['2014-03', 'yhat'].values))
print(nes_score1(r_s.loc['2015-03', 'Qi'].values, r_s.loc['2015-03', 'yhat'].values))
print(nes_score1(r_s.loc['2016-03', 'Qi'].values, r_s.loc['2016-03', 'yhat'].values))
print(nes_score1(r_s.loc['2017-03', 'Qi'].values, r_s.loc['2017-03', 'yhat'].values))
print('04')
print(nes_score1(r_s.loc['2014-04', 'Qi'].values, r_s.loc['2014-04', 'yhat'].values))
print(nes_score1(r_s.loc['2015-04', 'Qi'].values, r_s.loc['2015-04', 'yhat'].values))
print(nes_score1(r_s.loc['2016-04', 'Qi'].values, r_s.loc['2016-04', 'yhat'].values))
print(nes_score1(r_s.loc['2017-04', 'Qi'].values, r_s.loc['2017-04', 'yhat'].values))


print('05')
print(nes_score1(r_s.loc['2014-05', 'Qi'].values, r_s.loc['2014-05', 'yhat'].values))
print(nes_score1(r_s.loc['2015-05', 'Qi'].values, r_s.loc['2015-05', 'yhat'].values))
print(nes_score1(r_s.loc['2016-05', 'Qi'].values, r_s.loc['2016-05', 'yhat'].values))
print(nes_score1(r_s.loc['2017-05', 'Qi'].values, r_s.loc['2017-05', 'yhat'].values))
print(nes_score1(r_s.loc['2019-05', 'Qi'].values, r_s.loc['2019-05', 'yhat'].values))
print('06')

print(nes_score1(r_s.loc['2014-06', 'Qi'].values, r_s.loc['2014-06', 'yhat'].values))
print(nes_score1(r_s.loc['2015-06', 'Qi'].values, r_s.loc['2015-06', 'yhat'].values))
print(nes_score1(r_s.loc['2016-06', 'Qi'].values, r_s.loc['2016-06', 'yhat'].values))
print(nes_score1(r_s.loc['2017-06', 'Qi'].values, r_s.loc['2017-06', 'yhat'].values))
print('07')
print(nes_score1(r_s.loc['2014-07', 'Qi'].values, r_s.loc['2014-07', 'yhat'].values))
print(nes_score1(r_s.loc['2015-07', 'Qi'].values, r_s.loc['2015-07', 'yhat'].values))
print(nes_score1(r_s.loc['2016-07', 'Qi'].values, r_s.loc['2016-07', 'yhat'].values))
print(nes_score1(r_s.loc['2017-07', 'Qi'].values, r_s.loc['2017-07', 'yhat'].values))
print(nes_score1(r_s.loc['2018-07', 'Qi'].values, r_s.loc['2018-07', 'yhat'].values))
print(nes_score1(r_s.loc['2019-07', 'Qi'].values, r_s.loc['2019-07', 'yhat'].values))

print(nes_score1(r_s.loc['2014-08', 'Qi'].values, r_s.loc['2014-08', 'yhat'].values))
print(nes_score1(r_s.loc['2015-08', 'Qi'].values, r_s.loc['2015-08', 'yhat'].values))
print(nes_score1(r_s.loc['2016-08', 'Qi'].values, r_s.loc['2016-08', 'yhat'].values))
print(nes_score1(r_s.loc['2017-08', 'Qi'].values, r_s.loc['2017-08', 'yhat'].values))

print('10')
print(nes_score1(r_s.loc['2014-10', 'Qi'].values, r_s.loc['2014-10', 'yhat'].values))
print(nes_score1(r_s.loc['2015-10', 'Qi'].values, r_s.loc['2015-10', 'yhat'].values))
print(nes_score1(r_s.loc['2016-10', 'Qi'].values, r_s.loc['2016-10', 'yhat'].values))
print(nes_score1(r_s.loc['2017-10', 'Qi'].values, r_s.loc['2017-10', 'yhat'].values))
print(nes_score1(r_s.loc['2018-10', 'Qi'].values, r_s.loc['2018-10', 'yhat'].values))
print('11')
print(nes_score1(r_s.loc['2014-11', 'Qi'].values, r_s.loc['2014-11', 'yhat'].values))
print(nes_score1(r_s.loc['2015-11', 'Qi'].values, r_s.loc['2015-11', 'yhat'].values))
print(nes_score1(r_s.loc['2016-11', 'Qi'].values, r_s.loc['2016-11', 'yhat'].values))
print(nes_score1(r_s.loc['2017-11', 'Qi'].values, r_s.loc['2017-11', 'yhat'].values))

In [ ]:
test_y_hat_1 = lgb_df.loc['2018-02-01':'2018-02-07', ['yhat']].values
test_y_hat_2 = lgb_df.loc['2018-08-01':'2018-08-07', ['yhat']].values
test_y_hat_3 = lgb_df.loc['2018-11-01':'2018-11-07', ['yhat']].values

test_y_hat_4 = lgb_df.loc['2019-02-01':'2019-02-07', ['yhat']].values
test_y_hat_5 = lgb_df.loc['2019-04-01':'2019-04-07', ['yhat']].values
test_y_hat_6 = lgb_df.loc['2019-06-01':'2019-06-07', ['yhat']].values
test_y_hat_7 = lgb_df.loc['2019-08-01':'2019-08-07', ['yhat']].values
test_y_hat_8 = lgb_df.loc['2019-11-01':'2019-11-07', ['yhat']].values
#plt.plot(test_y_hat_1)
#plt.plot(test_y_hat_2)
#plt.plot(test_y_hat_3)
plt.plot(test_y_hat_4)
plt.plot(test_y_hat_5)
plt.plot(test_y_hat_6)
plt.plot(test_y_hat_7)
plt.plot(test_y_hat_8)
# submit
SubmitRaw = data_file + '/submission.csv'
submit_empty = pd.read_csv(SubmitRaw, index_col=0)

test_all_submission = np.vstack((test_y_hat_1.T, test_y_hat_2.T, test_y_hat_3.T))
submit_df = pd.DataFrame(test_all_submission, columns=submit_empty.columns)
submit_df.to_csv("./submission_fb.csv")

test_all_submission_final = np.vstack((test_y_hat_4.T, test_y_hat_5.T, test_y_hat_6.T, test_y_hat_7.T, test_y_hat_8.T))
submit_df_final = pd.DataFrame(test_all_submission_final, columns=submit_empty.columns)
submit_df_final.to_csv("./submission_fb_final.csv")

In [ ]:
lgb_df[['Qi', 'yhat']].plot()
print(lgb_df.tail(10))

In [ ]:
# engineering for lightgbm lightgbm t特征工程
# 平方和开方


# # for below features ,we have to shift, because the last 7 days has no data
# pf_df[['T_x_lag']] =pf_df[['T']].rolling(99).mean().shift(56) #we have
# pf_df[['w_x_lag']] =pf_df[['w']].rolling(65).mean().shift(56)

# #！ wd corr is too small
# lgb_df['T_x_lag_p'] = lgb_df['T_x_lag']**2
# lgb_df['T_x_lag_s'] = lgb_df['T_x_lag']**(1 / 2)
# lgb_df['rain_sum_x_lag_p'] = lgb_df['rain_sum_x_lag']**2
# lgb_df['rain_sum_x_lag_s'] = lgb_df['rain_sum_x_lag']**(1 / 2)
# lgb_df['w_x_lag_p'] = lgb_df['w_x_lag']**2
# lgb_df['w_x_lag_s'] = lgb_df['w_x_lag']**(1 / 2)
# lgb_df['future_5D_rain_sum_x_lag_p'] = lgb_df['future_5D_rain_sum_x_lag']**2
# lgb_df['future_5D_rain_sum_x_lag_s'] = lgb_df['future_5D_rain_sum_x_lag']**(1 / 2)

# lgb_df['w_rain_x_lag']=lgb_df['rain_sum_x_lag']*lgb_df['w_x_lag']
# lgb_df['w_T_x_lag']=lgb_df['T_x_lag']*lgb_df['w_x_lag']
# lgb_df['T_rain_x_lag']=lgb_df['rain_sum_x_lag']*lgb_df['T_x_lag']


target = 'Qi'  # 目标
cols = [
    'Qi',
    'T',
    'rain_sum',
    'w',
    'wd',
    'D1', 'D2', 'D3', 'D4', 'D5', 'future_5D_rain_sum'  # 以下列需要创建lag
]
non_use_cols = ['ds', target] + cols  # 这些列无法放入特征，因为预测当天没有这些数据


def create_lags(df, cols, lag, base=1, suffix=None):
    if suffix is None:
        suffix = '_' + str(lag) + 'days'
    for col in cols:
        df[col + suffix] = df[col].shift(lag * base)
    return df


lgb_df = create_lags(
    lgb_df,
    ['Qi'],
    lag=1,
    base=8,
    suffix=None)  # create 1 days lag  如果创建了小于7天的lag，那么需要用滚动预测

lgb_df = create_lags(
    lgb_df,
    ['Qi'],
    lag=1,
    base=1,
    suffix='3H')  # create 3H lag 如果创建了小于7天的lag，那么需要用滚动预测


# lgb_df = create_lags(
#     lgb_df,
#     cols,
#     lag=5,
#     base=8,
#     suffix=None)  # create 5 days lag

# lgb_df = create_lags(
#     lgb_df,
#     cols,
#     lag=7,
#     base=8,
#     suffix=None)  # create 7 days lag
# lgb_df = create_lags(lgb_df, cols, lag=30, base=8,
#                      suffix=None)  # create 30 days lag

# create time series period features #时间相关的特征
def time_period_features(df, time_col):
    # in hour level
    df['hour'] = df[time_col].dt.hour
    # in day level
    df['days_in_month'] = df[time_col].dt.days_in_month
    df['dayofyear'] = df[time_col].dt.dayofyear
    # week levels
    df['weekofyear'] = df[time_col].dt.weekofyear
    # month level
    df['month'] = df[time_col].dt.month
    # quarter level
    df['quarter'] = df[time_col].dt.quarter
    # year level
    df['year'] = df[time_col].dt.year
    return df


print(lgb_df.columns)

lgb_df = time_period_features(lgb_df, 'ds')

# #create window period features 滚动窗口特征，这里只有均值（有些求和），可能需要加std,median 等
# lgb_df[['T_7days_mean',
#         'rain_sum_7days_sum',
#         'w_7days_mean',
#         'wd_7days_mean',
#         'T_p_7days_mean',
#         'rain_sum_p_7days_sum',
#         'w_p_7days_mean',
#         'T_s_7days_mean',
#         'rain_sum__s_7days_sum',
#         'w_s_7days_mean',
#         ]] = lgb_df.rolling('12D').agg({'T_5days': np.mean,
#                                         'rain_sum_5days': np.sum,
#                                         'w_5days': np.mean,
#                                         'wd_5days': np.mean,
#                                         'T_p_5days': np.mean,
#                                         'rain_sum_p_5days': np.sum,
#                                         'w_p_5days': np.mean,
#                                         'T_s_5days': np.mean,
#                                         'rain_sum_s_5days': np.sum,
#                                         'w_s_5days': np.mean,
#                                         }).shift(12 * 8)
# lgb_df[['T_30days_mean',
#         'rain_sum_30days_sum',
#         'w_30days_mean',
#         'wd_30days_mean',
#         'T_p_30days_mean',
#         'rain_sum_p_30days_sum',
#         'w_p_30days_mean',
#         'T_s_30days_mean',
#         'rain_sum__s_30days_sum',
#         'w_s_30days_mean',
#         ]] = lgb_df.rolling('35D').agg({'T_5days': np.mean,
#                                         'rain_sum_5days': np.sum,
#                                         'w_5days': np.mean,
#                                         'wd_5days': np.mean,
#                                         'T_p_5days': np.mean,
#                                         'rain_sum_p_5days': np.sum,
#                                         'w_p_5days': np.mean,
#                                         'T_s_5days': np.mean,
#                                         'rain_sum_s_5days': np.sum,
#                                         'w_s_5days': np.mean,
#                                         }).shift(30 * 8)

In [ ]:
lgb_df.columns

In [ ]:
lgb_df[['D1_x_lag_std']] = lgb_df[['D1']].rolling(102).std()
lgb_df[['D2_x_lag_std']] = lgb_df[['D2']].rolling(110).std()
lgb_df[['D3_x_lag_std']] = lgb_df[['D3']].rolling(108).std()
lgb_df[['D4_x_lag_std']] = lgb_df[['D4']].rolling(123).std()
lgb_df[['D5_x_lag_std']] = lgb_df[['D5']].rolling(134).std()
lgb_df['future_5D_rain_sum_x_lag_std'] = lgb_df[['future_5D_rain_sum']].rolling(100).std()
lgb_df[['rain_pred_x_lag_std']] = lgb_df[['rain_pred']].rolling('11D6H').std()

In [ ]:
# rolling_cols = ['Qi3H']
# lgb_df[[c +'_30d_std' for c in rolling_cols]] =lgb_df[rolling_cols].rolling('30D3H').std().shift(30 * 8)
# lgb_df[[c +'_7d_std' for c in rolling_cols]] =lgb_df[rolling_cols].rolling('7D3H').std().shift(7 * 8)
# lgb_df[[c +'_1d_std' for c in rolling_cols]] =lgb_df[rolling_cols].rolling('1D3H').std().shift(1 * 8)
# lgb_df[[c +'_30d_median' for c in rolling_cols]] =lgb_df[rolling_cols].rolling('35D').median().shift(30 * 8)
# lgb_df[[c +'_7d_median' for c in rolling_cols]] =lgb_df[rolling_cols].rolling('12D').median().shift(7 * 8)
# lgb_df[[c +'_30d_max' for c in rolling_cols]] =lgb_df[rolling_cols].rolling('35D').max().shift(30 * 8)
# lgb_df[[c +'_7d_max' for c in rolling_cols]] =lgb_df[rolling_cols].rolling('12D').max().shift(7 * 8)
# lgb_df[[c +'_30d_min' for c in rolling_cols]] =lgb_df[rolling_cols].rolling('35D').min().shift(30 * 8)
# lgb_df[[c +'_7d_min' for c in rolling_cols]] =lgb_df[rolling_cols].rolling('12D').min().shift(7 * 8)

In [ ]:
lgb_df['2016-12'].isna().sum() # 训练集


In [ ]:
lgb_use_cols = ['rain_pred', 'D1_x_lag',
                'D2_x_lag', 'D3_x_lag', 'D4_x_lag', 'D5_x_lag',
                'future_5D_rain_sum_x_lag', 'rain_pred_x_lag',
                'on_season', 'off_season', 'D1_on', 'D1_off', 'D2_on', 'D2_off',
                'D3_on', 'D3_off', 'D4_on', 'D4_off', 'D5_on', 'D5_off', 'rain_pred_on',
                'rain_pred_off', 'yhat', 'D1_x_lag_std', 'D2_x_lag_std', 'D3_x_lag_std',
                'D4_x_lag_std', 'D5_x_lag_std', 'future_5D_rain_sum_x_lag_std',
                'rain_pred_x_lag_std', 'Qi_1days', 'Qi3H', 'hour', 'days_in_month',
                'dayofyear', 'weekofyear', 'month', 'quarter', 'year', 'Qi_diff']
non_use_cols = [x for x in list(lgb_df.columns) if x not in lgb_use_cols]
# split_train/test
train_df = lgb_df['2013-01':'2016-12'].copy()  # 训练集
validate_df = lgb_df['2017-01':'2019-11'].copy()  # 验证集
test_df_1 = lgb_df['2018-02-01':'2018-02-07']  # 测试1
test_df_2 = lgb_df['2018-08-01':'2018-08-07']  # 测试2
test_df_3 = lgb_df['2018-11-01':'2018-11-07']  # 测试3
test_df_4 = lgb_df['2019-02-01':'2019-02-07']  # 测试1
test_df_5 = lgb_df['2019-04-01':'2019-04-07']  # 测试2
test_df_6 = lgb_df['2019-06-01':'2019-06-07']  # 测试3
test_df_7 = lgb_df['2019-08-01':'2019-08-07']  # 测试2
test_df_8 = lgb_df['2019-11-01':'2019-11-07']  # 测试3
print(non_use_cols)
submit_train_df = lgb_df['2013-01':'2019-11'].copy()  # 合并训练集和验证集，用于最终提交时训练的数据


def remove_calirate_samples_by_col(df, col, window='1D', threshold=0.0):
    std = df[col].rolling(window=window).std()
    std.fillna(0, inplace=True)
    df = df[std > threshold]
    return df


train_df = remove_calirate_samples_by_col(train_df, 'Qi')
validate_df = remove_calirate_samples_by_col(validate_df, 'Qi')
submit_train_df = remove_calirate_samples_by_col(submit_train_df, 'Qi')

submit_train_df.dropna(axis=0, inplace=True)
submit_train_X = submit_train_df.drop(non_use_cols, axis=1)
submit_train_y = submit_train_df[target]

train_df.dropna(axis=0, inplace=True)  # 去掉包含nan的行（样本）
train_X = train_df.drop(non_use_cols, axis=1)
train_y = train_df[target]
validate_df.dropna(axis=0, inplace=True)  # 去掉包含nan的行（样本）
validate_X = validate_df.drop(non_use_cols, axis=1)
validate_y = validate_df[target]

print(submit_train_X.tail(66))
print(submit_train_X.shape)

print(submit_train_X.columns)


def nes_metric(y_ture, y_pred):  # 定义nes ，这里没有采用权重，是因为验证的数据集不一定是56个时间戳
    if isinstance(y_pred, lgb.Dataset):
        y_pred = y_pred.get_label()
    nes_score = 1 - np.sum((y_ture - y_pred) ** 2) / \
                np.sum((y_ture - np.mean(y_ture)) ** 2)
    return 'nes_score', nes_score * 100, True


test_X_1 = test_df_1.drop(non_use_cols, axis=1)
test_X_2 = test_df_2.drop(non_use_cols, axis=1)
test_X_3 = test_df_3.drop(non_use_cols, axis=1)
test_X_4 = test_df_4.drop(non_use_cols, axis=1)
test_X_5 = test_df_5.drop(non_use_cols, axis=1)
test_X_6 = test_df_6.drop(non_use_cols, axis=1)
test_X_7 = test_df_7.drop(non_use_cols, axis=1)
test_X_8 = test_df_8.drop(non_use_cols, axis=1)

print(test_X_1['yhat'])

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=0, n_jobs=-1)
rf.fit(submit_train_X.values, submit_train_y.values)
importances = rf.feature_importances_

In [ ]:
importance_s = pd.Series(importances, index=submit_train_X.columns)
importance_s.sort_values(ascending=False, inplace=True)
importance_s.head(20)

In [ ]:
# initial trial
dtrain = lgb.Dataset(train_X.values, label=train_y.values)
dvalidate = lgb.Dataset(validate_X.values, label=validate_y.values)
dtrain_submit = lgb.Dataset(submit_train_X.values, label=submit_train_y.values)
param = {
    'max_depth': 5,
    'num_leaves': 30,
    'eta': 0.05,
    'objective': 'regression',
    "sub_row": 0.5,
    "bagging_freq": 1,
    "lambda_l2": 0.3,
    'verbose': 0,
    'metric': None,
    'boosting': 'dart'
}
evals_result = {}
valid_sets = [dtrain, dvalidate]
valid_name = ['train', 'eval']
feature_name = list(train_X.columns)

cv_results = lgb.cv(  # kfold 验证
    param,
    dtrain_submit,
    num_boost_round=515,
    feature_name=feature_name,
    feval=nes_metric,
    stratified=False,
    verbose_eval=True,
    shuffle=False,
    nfold=5)

lgb_model = lgb.train(
    param,
    dtrain_submit,
    num_boost_round=515,
    feature_name=feature_name,
    feval=nes_metric,
    #valid_sets=valid_sets,
    #valid_names=valid_name,
    evals_result=evals_result)

In [ ]:
print(cv_results.keys())
fig, ax = plt.subplots()
ax.plot(cv_results['nes_score-mean'])
ax.plot(cv_results['nes_score-stdv'])
ax.set_ylim(0, 100)

In [ ]:
def recursive_forecast(model,test_X):
    test_X_1 = test_X.copy()
    for next_x in test_X_1.index:
        pred = model.predict(test_X_1.loc[[next_x]])
        if next_x + pd.Timedelta('3H') <= test_X_1.index.max():
            test_X_1.loc[[next_x + pd.Timedelta('3H')], ['Qi3H']] = pred
        if next_x + pd.Timedelta('1D') <= test_X_1.index.max():
            test_X_1.loc[[next_x + pd.Timedelta('1D')], ['Qi_1days']] = pred
    #         if next_x + pd.Timedelta('5D') <= test_X_1.index.max():
    #             test_X_1.loc[[next_x + pd.Timedelta('5D')], ['Qi_5days']] = pred
    return model.predict(test_X_1)


valie_y_hat_1 = lgb_model.predict(validate_X)
plt.plot(valie_y_hat_1)
plt.plot(validate_y.values)
nes_metric(validate_y.values, valie_y_hat_1)

valie_y_hat_1 = lgb_model.predict(validate_X)
plt.plot(valie_y_hat_1)
plt.plot(validate_y.values)
nes_metric(validate_y.values, valie_y_hat_1)

In [ ]:
def nes_score1(y_ture, y_pred):
    nes_score = 1 - np.sum((y_ture - y_pred) ** 2) / np.sum((y_ture - np.mean(y_ture)) ** 2)
    score = nes_score * 100
    return score


lgb_y = lgb_model.predict(submit_train_X.values)
lgb_y_df = pd.DataFrame(lgb_y, columns=['lgb_yhat'], index=submit_train_y.index)
if 'lgb_yhat' in lgb_df.columns:
    lgb_df['lgb_yhat'] = lgb_y_df['lgb_yhat']
else:
    lgb_df = pd.concat([lgb_df, lgb_y_df], axis=1)
r_s = lgb_df[['Qi', 'lgb_yhat']].copy().dropna(axis=0)
r_s = r_s.loc[:, ~r_s.columns.duplicated(keep='last')]

plt.plot(r_s['lgb_yhat'].values[-56:])
plt.plot(r_s['Qi'].values[-56:])
print(nes_score1(r_s['Qi'].values, r_s['lgb_yhat'].values))
print('01')
print(nes_score1(r_s.loc['2014-01', 'Qi'].values, r_s.loc['2014-01', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2015-01', 'Qi'].values, r_s.loc['2015-01', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2016-01', 'Qi'].values, r_s.loc['2016-01', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2017-01', 'Qi'].values, r_s.loc['2017-01', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2018-01', 'Qi'].values, r_s.loc['2018-01', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2019-01', 'Qi'].values, r_s.loc['2019-01', 'lgb_yhat'].values))

print('02')
print(nes_score1(r_s.loc['2014-02', 'Qi'].values, r_s.loc['2014-02', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2015-02', 'Qi'].values, r_s.loc['2015-02', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2016-02', 'Qi'].values, r_s.loc['2016-02', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2017-02', 'Qi'].values, r_s.loc['2017-02', 'lgb_yhat'].values))

print('03')
print(nes_score1(r_s.loc['2014-03', 'Qi'].values, r_s.loc['2014-03', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2015-03', 'Qi'].values, r_s.loc['2015-03', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2016-03', 'Qi'].values, r_s.loc['2016-03', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2017-03', 'Qi'].values, r_s.loc['2017-03', 'lgb_yhat'].values))
print('04')
print(nes_score1(r_s.loc['2014-04', 'Qi'].values, r_s.loc['2014-04', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2015-04', 'Qi'].values, r_s.loc['2015-04', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2016-04', 'Qi'].values, r_s.loc['2016-04', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2017-04', 'Qi'].values, r_s.loc['2017-04', 'lgb_yhat'].values))


print('05')
print(nes_score1(r_s.loc['2014-05', 'Qi'].values, r_s.loc['2014-05', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2015-05', 'Qi'].values, r_s.loc['2015-05', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2016-05', 'Qi'].values, r_s.loc['2016-05', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2017-05', 'Qi'].values, r_s.loc['2017-05', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2019-05', 'Qi'].values, r_s.loc['2019-05', 'lgb_yhat'].values))
print('06')

print(nes_score1(r_s.loc['2014-06', 'Qi'].values, r_s.loc['2014-06', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2015-06', 'Qi'].values, r_s.loc['2015-06', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2016-06', 'Qi'].values, r_s.loc['2016-06', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2017-06', 'Qi'].values, r_s.loc['2017-06', 'lgb_yhat'].values))
print('07')
print(nes_score1(r_s.loc['2014-07', 'Qi'].values, r_s.loc['2014-07', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2015-07', 'Qi'].values, r_s.loc['2015-07', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2016-07', 'Qi'].values, r_s.loc['2016-07', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2017-07', 'Qi'].values, r_s.loc['2017-07', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2018-07', 'Qi'].values, r_s.loc['2018-07', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2019-07', 'Qi'].values, r_s.loc['2019-07', 'lgb_yhat'].values))

print(nes_score1(r_s.loc['2014-08', 'Qi'].values, r_s.loc['2014-08', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2015-08', 'Qi'].values, r_s.loc['2015-08', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2016-08', 'Qi'].values, r_s.loc['2016-08', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2017-08', 'Qi'].values, r_s.loc['2017-08', 'lgb_yhat'].values))

print('10')
print(nes_score1(r_s.loc['2014-10', 'Qi'].values, r_s.loc['2014-10', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2015-10', 'Qi'].values, r_s.loc['2015-10', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2016-10', 'Qi'].values, r_s.loc['2016-10', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2017-10', 'Qi'].values, r_s.loc['2017-10', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2018-10', 'Qi'].values, r_s.loc['2018-10', 'lgb_yhat'].values))
print('11')
print(nes_score1(r_s.loc['2014-11', 'Qi'].values, r_s.loc['2014-11', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2015-11', 'Qi'].values, r_s.loc['2015-11', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2016-11', 'Qi'].values, r_s.loc['2016-11', 'lgb_yhat'].values))
print(nes_score1(r_s.loc['2017-11', 'Qi'].values, r_s.loc['2017-11', 'lgb_yhat'].values))

In [ ]:
print('01')
train_y_hat = recursive_forecast(lgb_model,train_X['2014-01'])
print(nes_metric(train_y['2014-01'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2015-01'])
print(nes_metric(train_y['2015-01'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2016-01'])
print(nes_metric(train_y['2016-01'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,validate_X['2017-01'])
print(nes_metric(validate_y['2017-01'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,validate_X['2018-01'])
print(nes_metric(validate_y['2018-01'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,validate_X['2019-01'])
print(nes_metric(validate_y['2019-01'].values, train_y_hat))
print('02')

train_y_hat = recursive_forecast(lgb_model,train_X['2014-02'])
print(nes_metric(train_y['2014-02'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2015-02'])
print(nes_metric(train_y['2015-02'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2016-02'])
print(nes_metric(train_y['2016-02'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,validate_X['2017-02'])
print(nes_metric(validate_y['2017-02'].values, train_y_hat))


print('03')
train_y_hat = recursive_forecast(lgb_model,train_X['2014-03'])
print(nes_metric(train_y['2014-03'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2015-03'])
print(nes_metric(train_y['2015-03'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2016-03'])
print(nes_metric(train_y['2016-03'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,validate_X['2017-03'])
print(nes_metric(validate_y['2017-03'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,validate_X['2019-03'])
print(nes_metric(validate_y['2019-03'].values, train_y_hat))
print('04')

train_y_hat = recursive_forecast(lgb_model,train_X['2014-04'])
print(nes_metric(train_y['2014-04'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2015-04'])
print(nes_metric(train_y['2015-04'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2016-04'])
print(nes_metric(train_y['2016-04'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,validate_X['2017-04'])
print(nes_metric(validate_y['2017-04'].values, train_y_hat))



print('05')
train_y_hat = recursive_forecast(lgb_model,train_X['2014-05'])
print(nes_metric(train_y['2014-05'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2015-05'])
print(nes_metric(train_y['2015-05'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2016-05'])
print(nes_metric(train_y['2016-05'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,validate_X['2017-05'])
print(nes_metric(validate_y['2017-05'].values, train_y_hat))
#train_y_hat = recursive_forecast(lgb_model,validate_X['2019-05'])
#print(nes_metric(validate_y['2019-05'].values, train_y_hat))
print('06')

train_y_hat = recursive_forecast(lgb_model,train_X['2014-06'])
print(nes_metric(train_y['2014-06'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2015-06'])
print(nes_metric(train_y['2015-06'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2016-06'])
print(nes_metric(train_y['2016-06'].values, train_y_hat))
#train_y_hat = recursive_forecast(lgb_model,validate_X['2017-06'])
#print(nes_metric(validate_y['2017-06'].values, train_y_hat))   # [DatetimeIndex(['2017-06-05 20:00:00'], dtype='datetime64[ns]', freq=None)] are in the [index]"


print('07')
train_y_hat = recursive_forecast(lgb_model,train_X['2014-07'])
print(nes_metric(train_y['2014-07'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2015-07'])
print(nes_metric(train_y['2015-07'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2016-07'])
print(nes_metric(train_y['2016-07'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,validate_X['2017-07'])
print(nes_metric(validate_y['2017-07'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,validate_X['2019-07'])
print(nes_metric(validate_y['2019-07'].values, train_y_hat))
print('08')

train_y_hat = recursive_forecast(lgb_model,train_X['2014-08'])
print(nes_metric(train_y['2014-08'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2015-08'])
print(nes_metric(train_y['2015-08'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2016-08'])
print(nes_metric(train_y['2016-08'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,validate_X['2017-08'])
print(nes_metric(validate_y['2017-08'].values, train_y_hat))


print('10')
train_y_hat = recursive_forecast(lgb_model,train_X['2014-10'])
print(nes_metric(train_y['2014-10'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2015-10'])
print(nes_metric(train_y['2015-10'].values, train_y_hat))
#train_y_hat = recursive_forecast(lgb_model,train_X['2016-10']) #[DatetimeIndex(['2016-10-05 02:00:00'], dtype='datetime64[ns]', freq=None)] are in the [index]"
#print(nes_metric(train_y['2016-10'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,validate_X['2017-10'])
print(nes_metric(validate_y['2017-10'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,validate_X['2019-10'])
print(nes_metric(validate_y['2019-10'].values, train_y_hat))

print('11')


train_y_hat = recursive_forecast(lgb_model,train_X['2014-11'])
print(nes_metric(train_y['2014-11'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2015-11'])
print(nes_metric(train_y['2015-11'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,train_X['2016-11'])
print(nes_metric(train_y['2016-11'].values, train_y_hat))
train_y_hat = recursive_forecast(lgb_model,validate_X['2017-11'])
print(nes_metric(validate_y['2017-11'].values, train_y_hat))


In [ ]:
lgb.plot_importance(lgb_model)

In [ ]:
test_y_hat_1 = recursive_forecast(lgb_model,test_X_1)
test_y_hat_2 = recursive_forecast(lgb_model,test_X_2)
test_y_hat_3 = recursive_forecast(lgb_model,test_X_3)
test_y_hat_4 = recursive_forecast(lgb_model,test_X_4)
test_y_hat_5 = recursive_forecast(lgb_model,test_X_5)
test_y_hat_6 = recursive_forecast(lgb_model,test_X_6)
test_y_hat_7 = recursive_forecast(lgb_model,test_X_7)
test_y_hat_8 = recursive_forecast(lgb_model,test_X_8)

plt.plot(test_y_hat_1)
plt.plot(test_y_hat_2)
plt.plot(test_y_hat_3)
plt.plot(test_y_hat_4)
plt.plot(test_y_hat_5)
plt.plot(test_y_hat_6)
plt.plot(test_y_hat_7)
plt.plot(test_y_hat_8)
# submit
SubmitRaw = data_file + '/submission.csv'
submit_empty = pd.read_csv(SubmitRaw, index_col=0)

test_all_submission = np.vstack((test_y_hat_1, test_y_hat_2, test_y_hat_3))
submit_df = pd.DataFrame(test_all_submission, columns=submit_empty.columns)
submit_df.to_csv("./submission_lgb_2.csv")


test_all_submission_final = np.vstack((test_y_hat_4, test_y_hat_5, test_y_hat_6, test_y_hat_7, test_y_hat_8))
submit_df_final = pd.DataFrame(test_all_submission_final, columns=submit_empty.columns)
submit_df_final.to_csv("./submission_lgb_2_final.csv")

# add lgb predict to lgb_df


lgb_df.loc[test_X_1.index, 'lgb_yhat'] = test_y_hat_1
lgb_df.loc[test_X_2.index, 'lgb_yhat'] = test_y_hat_2
lgb_df.loc[test_X_3.index, 'lgb_yhat'] = test_y_hat_3
lgb_df.loc[test_X_4.index, 'lgb_yhat'] = test_y_hat_4
lgb_df.loc[test_X_5.index, 'lgb_yhat'] = test_y_hat_5
lgb_df.loc[test_X_6.index, 'lgb_yhat'] = test_y_hat_6
lgb_df.loc[test_X_7.index, 'lgb_yhat'] = test_y_hat_7
lgb_df.loc[test_X_8.index, 'lgb_yhat'] = test_y_hat_8

print(lgb_df.loc['2018-02-01':'2018-02-07', 'lgb_yhat'])

In [ ]:
lgb_df.loc['2019-02-01':'2019-02-07', 'lgb_yhat'].plot()
lgb_df.loc['2019-04-01':'2019-04-07', 'lgb_yhat'].plot()
lgb_df.loc['2019-06-01':'2019-06-07', 'lgb_yhat'].plot()

lgb_df.loc['2019-08-01':'2019-08-07', 'lgb_yhat'].plot()
lgb_df.loc['2019-11-01':'2019-11-07', 'lgb_yhat'].plot()

In [ ]:
dnn_df = lgb_df.copy()
dnn_df.columns

In [ ]:
# replace with standard DNN
from tensorflow.keras import regularizers
import keras.backend as K
from sklearn.utils import shuffle
from keras.layers import Dense, LSTM, Dropout
from keras.models import Sequential
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt

# split_train/test
train_df = dnn_df['2013-01':'2016-12'].copy()
validate_df = dnn_df['2017-01':'2019-10'].copy()
test_df_1 = dnn_df['2018-02-01':'2018-02-07']
test_df_2 = dnn_df['2018-08-01':'2018-08-07']
test_df_3 = dnn_df['2018-11-01':'2018-11-07']
test_df_4 = dnn_df['2019-02-01':'2019-02-07']
test_df_5 = dnn_df['2019-04-01':'2019-04-07']
test_df_6 = dnn_df['2019-06-01':'2019-06-07']
test_df_7 = dnn_df['2019-08-01':'2019-08-07']
test_df_8 = dnn_df['2019-11-01':'2019-11-07']


def nes_loss_1D(y_ture, y_pred):
    nse_ = 1 - K.sum((y_ture - y_pred) ** 2) / K.sum((y_ture - K.mean(y_ture)) ** 2)
    loss = (1 - nse_) * 100
    return loss


# select features to build X y
use_cols = ['D1', 'D2', 'D3', 'D4', 'D5', 'rain_pred', 'D1_x_lag',
            'D2_x_lag', 'D3_x_lag', 'D4_x_lag', 'D5_x_lag', 'rain_pred_x_lag', 'on_season', 'off_season', 'D1_on',
            'D1_off', 'D2_on', 'D2_off',
            'D3_on', 'D3_off', 'D4_on', 'D4_off', 'D5_on', 'D5_off', 'rain_pred_on',
            'rain_pred_off', 'yhat', 'hour', 'days_in_month',
            'dayofyear', 'weekofyear', 'month', 'quarter', 'year', 'D1_x_lag_std', 'D2_x_lag_std', 'D3_x_lag_std',
            'D4_x_lag_std',
            'D5_x_lag_std', 'future_5D_rain_sum_x_lag_std', 'rain_pred_x_lag_std',
            'lgb_yhat']

#use_cols = [ 'yhat','lgb_yhat','hour', 'days_in_month',            'dayofyear', 'weekofyear', 'month', 'quarter', 'year']

train_no_nan = train_df[use_cols + ['Qi']].dropna(axis=0)
valid_no_nan = validate_df[use_cols + ['Qi']].dropna(axis=0)
submit_no_nan = pd.concat([train_no_nan, valid_no_nan], axis=0)

train_X_raw = train_no_nan[use_cols].copy()
valid_X_raw = valid_no_nan[use_cols].copy()
submit_X_raw = submit_no_nan[use_cols].copy()
y_train_raw = train_no_nan[['Qi']].values
y_valid_raw = valid_no_nan[['Qi']].values
y_submit_raw = submit_no_nan[['Qi']].values

test_X_raw_1 = test_df_1[use_cols].copy()
test_X_raw_2 = test_df_2[use_cols].copy()
test_X_raw_3 = test_df_3[use_cols].copy()
test_X_raw_4 = test_df_4[use_cols].copy()
test_X_raw_5 = test_df_5[use_cols].copy()
test_X_raw_6 = test_df_6[use_cols].copy()
test_X_raw_7 = test_df_7[use_cols].copy()
test_X_raw_8 = test_df_8[use_cols].copy()
# scale the data
sc = MinMaxScaler(feature_range=(0, 1))
train_X_scaled = sc.fit_transform(train_X_raw.values)
valid_X_scaled = sc.transform(valid_X_raw.values)
submit_X_scaled = sc.transform(submit_X_raw.values)

test_X_scaled_1 = sc.transform(test_X_raw_1.values)
test_X_scaled_2 = sc.transform(test_X_raw_2.values)
test_X_scaled_3 = sc.transform(test_X_raw_3.values)
test_X_scaled_4 = sc.transform(test_X_raw_4.values)
test_X_scaled_5 = sc.transform(test_X_raw_5.values)
test_X_scaled_6 = sc.transform(test_X_raw_6.values)
test_X_scaled_7 = sc.transform(test_X_raw_7.values)
test_X_scaled_8 = sc.transform(test_X_raw_8.values)

dnn_model = Sequential()
dnn_model.add(
    Dense(10, input_shape=(train_X_scaled.shape[1],), kernel_initializer='glorot_uniform',
          kernel_regularizer=regularizers.l2(0.0)))
dnn_model.add(Dropout(0.1))
dnn_model.add(Dense(1))
dnn_model.compile(optimizer='adam', loss=nes_loss_1D)
print(dnn_model.summary())
train_X, train_y = shuffle(train_X_scaled, y_train_raw, random_state=42)
#train_X, train_y = shuffle(submit_X_scaled, y_submit_raw)#, random_state=42)


# fit dnn_model
history = dnn_model.fit(
    train_X,
    train_y,
    epochs=3000,
    batch_size=3000,validation_split =0.1)

figure,ax = plt.subplots()
ax.plot(history.history['loss'],
         'b',
         label='Training loss')
ax.plot(history.history['val_loss'],
         'r',
         label='Validation loss')
ax.set_ylim(0,100)
plt.show()

In [ ]:
def nes_score1(y_ture, y_pred):
    nes_score = 1 - np.sum((y_ture - y_pred) ** 2) / np.sum((y_ture - np.mean(y_ture)) ** 2)
    score = nes_score * 100
    return score


dnn_y = dnn_model.predict(submit_X_scaled)
dnn_yhat_df = pd.DataFrame(dnn_y, columns=['dnn_yhat'], index=submit_no_nan.index)
if 'dnn_yhat' in dnn_df.columns:
    dnn_df['dnn_yhat'] = dnn_yhat_df['dnn_yhat']
else:
    dnn_df = pd.concat([dnn_df, dnn_yhat_df], axis=1)
r_s = dnn_df[['Qi', 'dnn_yhat']].copy().dropna(axis=0)
r_s = r_s.loc[:, ~r_s.columns.duplicated()]

plt.plot(r_s['dnn_yhat'].values[-56:])
plt.plot(r_s['Qi'].values[-56:])
print(nes_score1(r_s['Qi'].values, r_s['dnn_yhat'].values))
print('01')
print(nes_score1(r_s.loc['2014-01', 'Qi'].values, r_s.loc['2014-01', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2015-01', 'Qi'].values, r_s.loc['2015-01', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2016-01', 'Qi'].values, r_s.loc['2016-01', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2017-01', 'Qi'].values, r_s.loc['2017-01', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2018-01', 'Qi'].values, r_s.loc['2018-01', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2019-01', 'Qi'].values, r_s.loc['2019-01', 'dnn_yhat'].values))

print('02')
print(nes_score1(r_s.loc['2014-02', 'Qi'].values, r_s.loc['2014-02', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2015-02', 'Qi'].values, r_s.loc['2015-02', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2016-02', 'Qi'].values, r_s.loc['2016-02', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2017-02', 'Qi'].values, r_s.loc['2017-02', 'dnn_yhat'].values))

print('03')
print(nes_score1(r_s.loc['2014-03', 'Qi'].values, r_s.loc['2014-03', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2015-03', 'Qi'].values, r_s.loc['2015-03', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2016-03', 'Qi'].values, r_s.loc['2016-03', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2017-03', 'Qi'].values, r_s.loc['2017-03', 'dnn_yhat'].values))
print('04')
print(nes_score1(r_s.loc['2014-04', 'Qi'].values, r_s.loc['2014-04', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2015-04', 'Qi'].values, r_s.loc['2015-04', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2016-04', 'Qi'].values, r_s.loc['2016-04', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2017-04', 'Qi'].values, r_s.loc['2017-04', 'dnn_yhat'].values))


print('05')
print(nes_score1(r_s.loc['2014-05', 'Qi'].values, r_s.loc['2014-05', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2015-05', 'Qi'].values, r_s.loc['2015-05', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2016-05', 'Qi'].values, r_s.loc['2016-05', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2017-05', 'Qi'].values, r_s.loc['2017-05', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2019-05', 'Qi'].values, r_s.loc['2019-05', 'dnn_yhat'].values))
print('06')

print(nes_score1(r_s.loc['2014-06', 'Qi'].values, r_s.loc['2014-06', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2015-06', 'Qi'].values, r_s.loc['2015-06', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2016-06', 'Qi'].values, r_s.loc['2016-06', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2017-06', 'Qi'].values, r_s.loc['2017-06', 'dnn_yhat'].values))
print('07')
print(nes_score1(r_s.loc['2014-07', 'Qi'].values, r_s.loc['2014-07', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2015-07', 'Qi'].values, r_s.loc['2015-07', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2016-07', 'Qi'].values, r_s.loc['2016-07', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2017-07', 'Qi'].values, r_s.loc['2017-07', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2018-07', 'Qi'].values, r_s.loc['2018-07', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2019-07', 'Qi'].values, r_s.loc['2019-07', 'dnn_yhat'].values))

print(nes_score1(r_s.loc['2014-08', 'Qi'].values, r_s.loc['2014-08', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2015-08', 'Qi'].values, r_s.loc['2015-08', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2016-08', 'Qi'].values, r_s.loc['2016-08', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2017-08', 'Qi'].values, r_s.loc['2017-08', 'dnn_yhat'].values))

print('10')
print(nes_score1(r_s.loc['2014-10', 'Qi'].values, r_s.loc['2014-10', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2015-10', 'Qi'].values, r_s.loc['2015-10', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2016-10', 'Qi'].values, r_s.loc['2016-10', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2017-10', 'Qi'].values, r_s.loc['2017-10', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2018-10', 'Qi'].values, r_s.loc['2018-10', 'dnn_yhat'].values))
print('11')
print(nes_score1(r_s.loc['2014-11', 'Qi'].values, r_s.loc['2014-11', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2015-11', 'Qi'].values, r_s.loc['2015-11', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2016-11', 'Qi'].values, r_s.loc['2016-11', 'dnn_yhat'].values))
print(nes_score1(r_s.loc['2017-11', 'Qi'].values, r_s.loc['2017-11', 'dnn_yhat'].values))

In [ ]:
#print(test_X_scaled_1)
# test set predict
test_y_hat_1 = dnn_model.predict(test_X_scaled_1)
test_y_hat_2 = dnn_model.predict(test_X_scaled_2)
test_y_hat_3 = dnn_model.predict(test_X_scaled_3)
test_y_hat_4 = dnn_model.predict(test_X_scaled_4)
test_y_hat_5 = dnn_model.predict(test_X_scaled_5)
test_y_hat_6 = dnn_model.predict(test_X_scaled_6)
test_y_hat_7 = dnn_model.predict(test_X_scaled_7)
test_y_hat_8 = dnn_model.predict(test_X_scaled_8)

# plt.plot(test_y_hat_1)
# plt.plot(test_y_hat_2)
#plt.plot(test_y_hat_3)
#plt.plot(test_y_hat_4)

plt.plot(test_y_hat_8)

# plt.plot(dnn_df.loc['2018-02-01':'2018-02-07',['yhat','lgb_yhat']].values)
# plt.plot(dnn_df.loc['2018-08-01':'2018-08-07',['yhat','lgb_yhat']].values)
#plt.plot(dnn_df.loc['2018-11-01':'2018-11-07', ['yhat', 'lgb_yhat']].values)
#plt.plot(dnn_df.loc['2019-02-01':'2019-02-07', ['yhat', 'lgb_yhat']].values)

plt.plot(dnn_df.loc['2019-11-01':'2019-11-07', ['yhat', 'lgb_yhat']].values)

# submit
SubmitRaw = data_file + '/submission.csv'
submit_empty = pd.read_csv(SubmitRaw, index_col=0)

test_all_submission = np.vstack((test_y_hat_1.T, test_y_hat_2.T, test_y_hat_3.T))
submit_df = pd.DataFrame(test_all_submission, columns=submit_empty.columns)
submit_df.to_csv("./submission_dnn2.csv")


test_all_submission_final = np.vstack((test_y_hat_4.T, test_y_hat_5.T, test_y_hat_6.T, test_y_hat_7.T, test_y_hat_8.T))
submit_df_final = pd.DataFrame(test_all_submission_final, columns=submit_empty.columns)
submit_df_final.to_csv("./submission_dnn2_final.csv")